In [119]:
import pandas as pd
#load two data files 
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load= "Resources/students_complete.csv"

#read the data from csv into a dataframe
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

In [120]:
#determine if there are missing values in the student data frame
student_data_df.count()

Student ID       39170
student_name     39170
gender           39170
grade            39170
school_name      39170
reading_score    39170
math_score       39170
dtype: int64

In [121]:
#see if any null data
school_data_df.isnull().sum()

School ID      0
school_name    0
type           0
size           0
budget         0
dtype: int64

In [122]:
student_data_df.isnull().sum()

Student ID       0
student_name     0
gender           0
grade            0
school_name      0
reading_score    0
math_score       0
dtype: int64

In [123]:
student_data_df.notnull().sum()

Student ID       39170
student_name     39170
gender           39170
grade            39170
school_name      39170
reading_score    39170
math_score       39170
dtype: int64

In [124]:
student_data_df.dtypes

Student ID        int64
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
dtype: object

In [125]:
#fix the student names to take out the extra words
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]
#need to convert the student name to a string (not an object) for replace to work 
for i in prefixes_suffixes:
    student_data_df["student_name"] = student_data_df["student_name"].str.replace(i,"")


In [126]:
#merge two dataframes
school_data_complete_df = pd.merge(student_data_df, school_data_df, on = ["school_name","school_name"])

In [127]:
#get student count, pick any column since they all have same number of entries - 39170
student_count = school_data_complete_df["Student ID"].count()

In [128]:
schools = school_data_complete_df["school_name"].unique()

In [129]:
school_count = len(schools)

In [130]:
total_budget = school_data_df["budget"].sum()

In [131]:
average_reading_score = school_data_complete_df["reading_score"].mean()

In [132]:
average_math_score = school_data_complete_df["math_score"].mean()

In [133]:
passing_math =school_data_complete_df[school_data_complete_df["math_score"]>=70]
passing_reading = school_data_complete_df[school_data_complete_df["reading_score"]>=70]
passing_math_count = passing_math["student_name"].count()

In [134]:
passing_reading_count = passing_reading["student_name"].count()

In [135]:
percent_passing_math = passing_math_count / student_data_df["student_name"].count() * 100

In [136]:
percent_passing_reading = passing_reading_count / student_data_df["student_name"].count()*100

In [137]:
passing_both = school_data_complete_df[(school_data_complete_df["math_score"]>=70) & (school_data_complete_df["reading_score"]>=70)]

passing_both_number = passing_both["student_name"].count()

In [138]:
passing_both_percent = passing_both_number/student_data_df["student_name"].count()*100

In [139]:
#create a new data frame

district_summary_df = pd.DataFrame([{"Total schools": school_count, "Total students": student_count, 
                                     "Total Budget": total_budget, "Average Math Score": average_math_score, 
                                    "Average Reading Score": average_reading_score, "%passing math": percent_passing_math,
                                    "%passing reading": percent_passing_reading, "%overall passing rate": passing_both_percent, 
                                    }])


In [140]:
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)

district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)

district_summary_df["%passing math"] = district_summary_df["%passing math"].map("{:.1f}".format)

district_summary_df["%passing reading"] = district_summary_df["%passing reading"].map("{:.1f}".format)

district_summary_df["%overall passing rate"] = district_summary_df["%overall passing rate"].map("{:.1f}".format)

district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)

district_summary_df["Total students"] = district_summary_df["Total students"].map("{:,}".format)

district_summary_df


,Total schools,Total students,Total Budget,Average Math Score,Average Reading Score,%passing math,%passing reading,%overall passing rate
0,15,"39,170","$24,649,428.00",79.0,81.9,75.0,85.8,65.2


In [141]:
#get data per school

per_school_types = school_data_df.set_index(["school_name"])["type"]

In [142]:
df = pd.DataFrame(per_school_types)

In [143]:
#wrong index on school data frame
per_school_counts = school_data_df.set_index(["school_name"])["size"]

In [144]:
per_school_budget = school_data_df.set_index(["school_name"])["budget"]

In [145]:
per_school_capita = per_school_budget / per_school_counts

In [146]:
per_school_math= school_data_complete_df.groupby(["school_name"]).mean()["math_score"]
per_school_math.dtype

dtype('float64')

In [147]:
per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]

In [148]:
#find percent passing math by school
per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]
per_school_passing_math = per_school_passing_math / per_school_counts*100

In [149]:
#find %passing reading by school 
per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"]>=70)]
per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]
per_school_passing_reading = per_school_passing_reading / per_school_counts *100

In [150]:
#get students who passed both math and reading 

per_passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"]>= 70) & (school_data_complete_df["reading_score"]>=70)]
per_passing_math_reading = per_passing_math_reading.groupby(["school_name"]).count()["student_name"]

In [151]:
per_overall_passing_percentage = per_passing_math_reading / per_school_counts* 100


In [152]:
#create the per school data frame

per_school_summary_df = pd.DataFrame({"School Type": per_school_types,
             "Total Students": per_school_counts,
             "Total School Budget": per_school_budget,
             "Per Student Budget": per_school_capita,
             "Average Math Score": per_school_math,
           "Average Reading Score": per_school_reading,
           "% Passing Math": per_school_passing_math,
           "% Passing Reading": per_school_passing_reading,
           "% Overall Passing": per_overall_passing_percentage})


In [153]:
#format the per school data frame

per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)

per_school_summary_df["Per Student Budget"] = per_school_summary_df["Per Student Budget"].map("${:,.2f}".format)

per_school_summary_df["% Passing Math"] = per_school_summary_df["% Passing Math"].map("{:.0f}".format)

per_school_summary_df["% Passing Reading"] = per_school_summary_df["% Passing Reading"].map("{:.0f}".format)

per_school_summary_df["% Overall Passing"] = per_school_summary_df["% Overall Passing"].map("{:.0f}".format)

per_school_summary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,67,82,55
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94,97,91
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,66,81,53
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68,79,54
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93,97,91
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,67,81,54
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,93,96,89
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,66,81,54
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66,81,54
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,95,96,91


In [154]:
#objects in some of these items may be causing errors later when trying to find the mean
per_school_summary_df["% Passing Math"] = pd.to_numeric(per_school_summary_df["% Passing Math"])
per_school_summary_df["% Passing Reading"] =pd.to_numeric(per_school_summary_df["% Passing Reading"])
per_school_summary_df["% Overall Passing"]=pd.to_numeric(per_school_summary_df["% Overall Passing"])


In [155]:
#sort by top schools

top_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=False)

In [156]:
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94,97,91
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93,97,91
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,95,96,91
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93,97,91
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,94,97,91


In [157]:
bottom_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending = True)

In [158]:
bottom_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,66,81,53
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66,80,53
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68,79,54
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,67,81,54
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,66,81,54


In [159]:
#get grade level info
ninth_graders = school_data_complete_df[school_data_complete_df["grade"] == "9th"]
tenth_graders = school_data_complete_df[school_data_complete_df["grade"] == "10th"]
eleventh_graders = school_data_complete_df[school_data_complete_df["grade"]=="11th"]
twelfth_graders = school_data_complete_df[school_data_complete_df["grade"]=="12th"]


In [160]:
#get math scores average by school 

ninth_grade_math_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]
tenth_grade_math_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]
eleventh_grade_math_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]
twelfth_grade_math_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]

In [161]:
#get reading score by school

ninth_grade_reading_score = ninth_graders.groupby(["school_name"]).mean()["reading_score"]
tenth_grade_reading_score = tenth_graders.groupby(["school_name"]).mean()["reading_score"]
eleventh_grade_reading_score = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]
twelfth_grade_reading_score = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]

In [162]:
#create a adataframe for school by grade
math_scores_by_grade = pd.DataFrame({"9th": ninth_grade_math_scores, "10th": tenth_grade_math_scores,
                                    "11th": eleventh_grade_math_scores, "12th": twelfth_grade_math_scores})
#format 
math_scores_by_grade["9th"] = math_scores_by_grade["9th"].map("{:.1f}".format)
math_scores_by_grade["10th"] = math_scores_by_grade["10th"].map("{:.1f}".format)
math_scores_by_grade["11th"] = math_scores_by_grade["11th"].map("{:.1f}".format)
math_scores_by_grade["12th"] = math_scores_by_grade["12th"].map("{:.1f}".format)

#remove index
math_scores_by_grade.index.name = None
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.1,77.0,77.5,76.5
Cabrera High School,83.1,83.2,82.8,83.3
Figueroa High School,76.4,76.5,76.9,77.2
Ford High School,77.4,77.7,76.9,76.2
Griffin High School,82.0,84.2,83.8,83.4
Hernandez High School,77.4,77.3,77.1,77.2
Holden High School,83.8,83.4,85.0,82.9
Huang High School,77.0,75.9,76.4,77.2
Johnson High School,77.2,76.7,77.5,76.9
Pena High School,83.6,83.4,84.3,84.1


In [163]:
#reading scores by grade data frame:

reading_scores_by_grade = pd.DataFrame({
              "9th": ninth_grade_reading_score,
              "10th": tenth_grade_reading_score,
              "11th": eleventh_grade_reading_score,
              "12th": twelfth_grade_reading_score})

reading_scores_by_grade["9th"]= reading_scores_by_grade["9th"].map("{:.1f}".format)
reading_scores_by_grade["10th"]= reading_scores_by_grade["10th"].map("{:.1f}".format)
reading_scores_by_grade["11th"]= reading_scores_by_grade["11th"].map("{:.1f}".format)
reading_scores_by_grade["12th"]= reading_scores_by_grade["12th"].map("{:.1f}".format)

reading_scores_by_grade.index.name = None
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.3,80.9,80.9,80.9
Cabrera High School,83.7,84.3,83.8,84.3
Figueroa High School,81.2,81.4,80.6,81.4
Ford High School,80.6,81.3,80.4,80.7
Griffin High School,83.4,83.7,84.3,84.0
Hernandez High School,80.9,80.7,81.4,80.9
Holden High School,83.7,83.3,83.8,84.7
Huang High School,81.3,81.5,81.4,80.3
Johnson High School,81.3,80.8,80.6,81.2
Pena High School,83.8,83.6,84.3,84.6


In [164]:
#sort the results into 4 spending bins
per_school_capita.describe()
spending_bins = [0,585,630,645,675]
group_names = ["<$584","$585-629","$630-644","$645-675"]

#add a new column to data frame per school summary
per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels = group_names)

In [165]:
#calculate averages by column for each bin 

spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

spending_summary_df = pd.DataFrame({"Average Math Score": spending_math_scores, 
                                   "Average Reading Score": spending_reading_scores,
                                   "% Passing Math": spending_passing_math,
                                   "% Passing Reading": spending_passing_reading,
                                   "% Overall Passing": overall_passing_spending})

#format the dataframe

spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)
spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)
spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}".format)
spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)
spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}".format)



spending_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.5,83.9,94,97,90
$585-629,81.9,83.2,87,93,82
$630-644,78.5,81.6,73,84,63
$645-675,77.0,81.0,66,81,54


In [166]:
#calculate the results by school size

size_bins = [0,1000,2000,5000]
group_names = ["Small <1000", "Medium 1000-2000", "Large 2000-5000"]
#add a column to the per school summary df using cut function 
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels = group_names)

#calculate the results per school size

size_math_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary_df.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary_df.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary_df.groupby(["School Size"]).mean()["% Overall Passing"]

#put into a new dataframe

size_summary_df = pd.DataFrame({"Average Math Score": size_math_scores, 
                               "Average Reading Score": size_reading_scores,
                               "% Passing Math": size_passing_math, 
                               "% Passing Reading": size_passing_reading,
                               "% Overall Passing": size_overall_passing})

#format

size_summary_df["Average Math Score"] = size_summary_df["Average Math Score"].map("{:.1f}".format)
size_summary_df["Average Reading Score"] = size_summary_df["Average Reading Score"].map("{:.1f}".format)
size_summary_df["% Passing Math"] = size_summary_df["% Passing Math"].map("{:.0f}".format)
size_summary_df["% Passing Reading"] = size_summary_df["% Passing Reading"].map("{:.0f}".format)
size_summary_df["% Overall Passing"] = size_summary_df["% Overall Passing"].map("{:.0f}".format)

size_summary_df




,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small <1000,83.8,83.9,94,96,90
Medium 1000-2000,83.4,83.9,93,97,91
Large 2000-5000,77.7,81.3,70,83,58


In [167]:
# calculate scores by school type

type_math_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = per_school_summary_df.groupby(["School Type"]).mean()["Average Reading Score"]
type_passing_math= per_school_summary_df.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = per_school_summary_df.groupby(["School Type"]).mean()["% Passing Reading"]
type_overall_passing = per_school_summary_df.groupby(["School Type"]).mean()["% Overall Passing"]

#create a data frame for above

type_summary_df = pd.DataFrame({"Average Math Score": type_math_scores, 
                               "Average Reading Score": type_reading_scores,
                               "% Passing Math": type_passing_math,
                               "% Passing Reading":type_passing_reading,
                               "% Overall Passing": type_overall_passing})

#format dataframe

type_summary_df["Average Math Score"] = type_summary_df["Average Math Score"].map("{:.1f}".format)
type_summary_df["Average Reading Score"] = type_summary_df["Average Reading Score"].map("{:.1f}".format)
type_summary_df["% Passing Math"] = type_summary_df["% Passing Math"].map("{:.0f}".format)
type_summary_df["% Passing Reading"] = type_summary_df["% Passing Reading"].map("{:.0f}".format)
type_summary_df["% Overall Passing"] = type_summary_df["% Overall Passing"].map("{:.0f}".format)

type_summary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.5,83.9,94,97,90
District,77.0,81.0,67,81,54
